In [ ]:
import subprocess
import sys

def install(package, upgrade=False):
    try:
        command = [sys.executable, "-m", "pip", "install"]
        if upgrade:
            command.append("--upgrade")
        command.append(package)
        subprocess.check_call(command)
    except subprocess.CalledProcessError as e:
        print(f"Error installing package {package}: {e}")

install("pip", upgrade=True)
install("setuptools", upgrade=True)

required_packages = [
    "uproot", "awkward", "coffea", "qkeras",
    "tensorflow-model-optimization", "umap-learn",
    "numpy", "pandas", "matplotlib", "scikit-learn",
    "minisom"
]

for package in required_packages:
    install(package)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Concatenate
from qkeras import QActivation, QConv2D, QDense, quantized_bits
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import glob
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, SpectralEmbedding
from sklearn.cluster import KMeans, DBSCAN
import umap
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import pairwise_distances
from scipy.stats import gaussian_kde
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.colors import LinearSegmentedColormap

np.random.seed(42)

# Evaluate operations immediately instead of with graph execution (causes issue if disabled)
tf.config.run_functions_eagerly(True)

In [ ]:
# Custom layers defined in the original Convolutional Autoencoder (CAE)
class KerasPaddingLayer(tf.keras.layers.Layer):
    def call(self, x):
        padding = tf.constant([[0, 0], [0, 1], [0, 1], [0, 0]]) #pad height and width with 1 row/column.
        return tf.pad(x, padding, mode='CONSTANT', constant_values=0)

    def compute_output_shape(self, input_shape):
        batch_size, height, width, channels = input_shape #increase height and width by 1 -- keeps other dimensions the same
        return (batch_size, height + 1, width + 1, channels)


class KerasMinimumLayer(tf.keras.layers.Layer):
    def __init__(self, saturation_value=1, **kwargs):
        super(KerasMinimumLayer, self).__init__(**kwargs)
        self.saturation_value = saturation_value

    def call(self, x):
        return tf.minimum(x, self.saturation_value) #cap values at saturation_value

    def compute_output_shape(self, input_shape):
        return input_shape


class KerasFloorLayer(tf.keras.layers.Layer):
    def call(self, x):
        return tf.math.floor(x) #round down each element

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
# Encoder model setup
class EncoderModelBuilder:
    @staticmethod
    def build_encoder_model():
        input_shape = (8, 8, 1) 
        condition_shape = (8,) 

        wafer_input = Input(shape=input_shape, name='Wafer_Input')
        condition_input = Input(shape=condition_shape, name='Condition_Input')

        x = QActivation(activation=quantized_bits(bits=8, integer=1), name='Input_Quantization')(wafer_input)
        x = KerasPaddingLayer()(x)
        x = QConv2D(
            filters=8, kernel_size=3, strides=2, padding='valid',
            kernel_quantizer=quantized_bits(bits=6, integer=0, keep_negative=1, alpha=1),
            bias_quantizer=quantized_bits(bits=6, integer=0, keep_negative=1, alpha=1),
            name='Conv2D'
        )(x)
        x = QActivation(activation=quantized_bits(bits=8, integer=1), name='Activation')(x)
        x = Flatten()(x)
        x = QDense(
            units=16,
            kernel_quantizer=quantized_bits(bits=6, integer=0, keep_negative=1, alpha=1),
            bias_quantizer=quantized_bits(bits=6, integer=0, keep_negative=1, alpha=1),
            name='Dense_Layer'
        )(x)
        x = QActivation(activation=quantized_bits(bits=9, integer=1), name='Latent_Quantization')(x)
        latent_output = x

        # Optional quantization steps -- this can further refine the latent vector
        bits_per_output = 9
        if bits_per_output > 0:
            n_integer_bits = 1
            n_decimal_bits = bits_per_output - n_integer_bits
            output_max_int_size = 1 << n_decimal_bits
            output_saturation_value = (1 << n_integer_bits) - 1. / (1 << n_decimal_bits)

            latent_output = KerasFloorLayer()(latent_output * output_max_int_size)
            latent_output = KerasMinimumLayer(saturation_value=output_saturation_value)(latent_output / output_max_int_size)

        latent_output = Concatenate(axis=1)([latent_output, condition_input])

        encoder = Model(inputs=[wafer_input, condition_input], outputs=latent_output, name='Encoder_Model')
        return encoder

encoder = EncoderModelBuilder.build_encoder_model()
encoder.summary()


# Load in the pre-trained weights from the original CAE (original weights in tf format)
encoder.load_weights('best-encoder-epoch-converted.h5')

In [ ]:
# Load ROOT Files
def load_root_files(file_limit=-1, selected_eLinks=-1):
    files = glob.glob('*.root')
    if file_limit > 0:
        files = files[:file_limit] 

    all_inputs, all_conditions = [], []
    tree_name = 'FloatingpointThreshold0DummyHistomaxDummynTuple/HGCalTriggerNtuple'

    for file_index, file in enumerate(files):
        print(f"Processing file {file_index + 1}/{len(files)}: {file}")
        try:
            with uproot.open(file) as root_file:
                if tree_name not in root_file:
                    raise ValueError(f"Tree '{tree_name}' not found in file '{file}'")
                tree = root_file[tree_name]

                # Branches needed from the TTree
                branches = [
                    "gen_pt", "wafer_layer", "wafer_eta", "wafer_waferv",
                    "wafer_waferu", "wafer_wafertype"
                ]
                branches.extend([f"wafer_CALQ{j}" for j in range(64)])
                branches.extend([f"wafer_AEin{j}" for j in range(64)])

                data = tree.arrays(branches, library="ak") #load branches into awkward array
                mask = (ak.mean(data["gen_pt"], axis=1) >= 0) & (ak.mean(data["gen_pt"], axis=1) <= 100000) #filter ensuring all gen_pt events are non-negative and have an upper threshold
                data = data[mask]

                print(f"Number of events processed: {len(data['gen_pt'])}")

                # Extract and normalize data from the awkward array
                layers = ak.to_numpy(ak.flatten(data["wafer_layer"]))
                eta = ak.to_numpy(ak.flatten(data["wafer_eta"])) / 3.1
                wafer_v = ak.to_numpy(ak.flatten(data["wafer_waferv"])) / 12
                wafer_u = ak.to_numpy(ak.flatten(data["wafer_waferu"])) / 12
                wafer_type = ak.to_numpy(ak.flatten(data["wafer_wafertype"])).astype(int)
                one_hot_wafertype = np.eye(np.max(wafer_type) + 1)[wafer_type] 

                sum_CALQ = np.sum([ak.to_numpy(ak.flatten(data[f"wafer_CALQ{j}"])) for j in range(64)], axis=0)
                sum_CALQ = np.log(sum_CALQ + 1) #apply log transformation

                # Stack AEin values into 8x8 inputs
                inputs = np.stack([ak.to_numpy(ak.flatten(data[f"wafer_AEin{j}"])) for j in range(64)], axis=-1)
                inputs = np.reshape(inputs, (-1, 8, 8))

                # Apply eLinks selection mask (should be 11)
                selection_mask = {
                    5: (layers <= 11) & (layers >= 5),
                    4: (layers == 7) | (layers == 11),
                    3: (layers == 13),
                    2: (layers < 7) | (layers > 13),
                    -1: (layers > 0)
                }[selected_eLinks]

                print(f"Selected eLinks: {selected_eLinks}")

                print("Applying the following selection mask:")
                if selected_eLinks == 5:
                    print("Mask: Layers between 5 and 11 (inclusive).")
                elif selected_eLinks == 4:
                    print("Mask: Layers 7 or 11.")
                elif selected_eLinks == 3:
                    print("Mask: Layer 13.")
                elif selected_eLinks == 2:
                    print("Mask: Layers outside 7 to 13 (i.e., less than 7 or greater than 13).")
                elif selected_eLinks == -1:
                    print("Mask: No filtering on layers (all layers > 0).")

                inputs = inputs[selection_mask]
                eta = eta[selection_mask]
                wafer_v = wafer_v[selection_mask]
                wafer_u = wafer_u[selection_mask]
                one_hot_wafertype = one_hot_wafertype[selection_mask]
                sum_CALQ = sum_CALQ[selection_mask]
                layers_normalized = (layers[selection_mask] - 1) / 46

                print("Unique layers after selection:", np.unique(layers[selection_mask]))
                print(f"Number of wafers after selection: {inputs.shape[0]}")

                # Stack all condition features into a single array
                conditions = np.hstack([
                    eta[:, np.newaxis], wafer_v[:, np.newaxis], wafer_u[:, np.newaxis],
                    one_hot_wafertype, sum_CALQ[:, np.newaxis], layers_normalized[:, np.newaxis]
                ])

                all_inputs.append(inputs)
                all_conditions.append(conditions)

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    if not all_inputs:
        raise ValueError("No data was loaded. Please check your ROOT files and tree names.")

    return np.concatenate(all_inputs), np.concatenate(all_conditions)

In [ ]:
# Calculate latent representations
batch_size = 128

inputs, conditions = load_root_files(file_limit=-1)
print(inputs.shape)
print(conditions.shape)

inputs = np.expand_dims(inputs, axis=-1) #expand dimensions to match expected shape (8, 8, 1)
latent_representations = encoder.predict([inputs, conditions], batch_size) 

In [ ]:
total_cond_dim = conditions.shape[1]

eta_index = 0
wafer_v_index = 1
wafer_u_index = 2
wafertype_start = 3
wafertype_end = wafertype_start + 3  # One-hot encoding has 3 columns
sumCALQ_index = wafertype_end
layers_index = sumCALQ_index + 1

eta = conditions[:, eta_index]
waferv = conditions[:, wafer_v_index]
waferu = conditions[:, wafer_u_index]
wafertype_onehot = conditions[:, wafertype_start:wafertype_end]
sumCALQ = conditions[:, sumCALQ_index]
layers = conditions[:, layers_index]

unique_blob_labels = np.unique([-1, *range(20)])  #add -1 to account for potential outliers
colors_palette = sns.color_palette('husl', len(unique_blob_labels))  #number of colors match the number of unique labels
color_map = {label: colors_palette[i] for i, label in enumerate(unique_blob_labels)}

def plot_latent_space(latent_sample, attribute_sample, attribute_name, technique_name, embedding, component_labels, blob_labels=None):
    plt.figure(figsize=(14, 10))

    if blob_labels is not None:
        unique_labels = np.unique(blob_labels)

        # Map blob labels to colors
        colors = [color_map.get(label, '#FF0000') for label in blob_labels]  #bright red for outliers

        scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=colors, s=20, alpha=0.8, edgecolor='k')
        for label in unique_labels:
            if label != -1:
                x_text = np.mean(embedding[blob_labels == label, 0])
                y_text = np.mean(embedding[blob_labels == label, 1])
                plt.text(x_text, y_text, f'Blob {label + 1}', fontsize=12, weight='bold',
                         bbox=dict(facecolor='white', alpha=0.7, edgecolor='black'))
            else:
                x_text = np.mean(embedding[blob_labels == label, 0])
                y_text = np.mean(embedding[blob_labels == label, 1])
                plt.text(x_text, y_text, f'Outlier', fontsize=12, weight='bold',
                         bbox=dict(facecolor='white', alpha=0.7, edgecolor='black'))

        plt.title(f'{technique_name} Projection of Latent Space with Clustering by {attribute_name}', fontsize=16)
        handles = [
            plt.Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10,
                       markerfacecolor='#FF0000', markeredgewidth=1.5, markeredgecolor='k') if label == -1 else
            plt.Line2D([0], [0], marker='o', color='w', label=f'Blob {label + 1}', markersize=10,
                       markerfacecolor=color_map[label], markeredgewidth=1.5, markeredgecolor='k')
            for label in unique_labels
        ]
        plt.legend(handles=handles, title='Blob Labels', loc='best')

    else:
        scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=attribute_sample, s=20, cmap='viridis', alpha=0.8, edgecolor='k')
        plt.title(f'{technique_name} Projection of Latent Space Colored by {attribute_name}', fontsize=16)
        plt.colorbar(scatter, label=attribute_name, shrink=0.8)

    plt.xlabel(component_labels[0], fontsize=14)
    plt.ylabel(component_labels[1], fontsize=14)
    plt.grid(visible=True, linestyle='--', linewidth=0.5)
    plt.show()

attributes = {
    'eta': eta,
    'wafer_v': waferv,
    'wafer_u': waferu,
    'wafertype': np.argmax(wafertype_onehot, axis=1),
    'sumCALQ': sumCALQ,
    'layers': layers
}

In [ ]:
## 16D Latent Space + Conditions ##

# Sample subset of latent representations
num_samples = 10000
sample_indices = np.random.choice(len(latent_representations), num_samples, replace=False)
latent_sample = latent_representations[sample_indices]
conditions_subset = conditions[sample_indices]

# Or use entire set
#latent_sample = latent_representations

# UMAP with DBSCAN
umap_reducer = umap.UMAP(n_neighbors=10, min_dist=0.2, n_components=2, metric='cosine', random_state=42) #change n_neighbors to emphasize a local or global set
umap_embedding = umap_reducer.fit_transform(latent_sample)
dbscan = DBSCAN(eps=1.0, min_samples=10)
dbscan_labels = dbscan.fit_predict(umap_embedding)

In [ ]:
## 16D Latent Space ##

# Sample subset of latent representations
num_samples = 10000
sample_indices = np.random.choice(len(latent_representations), num_samples, replace=False)
latent_sample = latent_representations[sample_indices]
conditions_subset = conditions[sample_indices]

# Extract only the core 16D latent space, excluding the conditional information
core_latent_sample = latent_sample[:, :16]

umap_embedding = umap.UMAP(n_neighbors=150, min_dist=0.2, n_components=2, metric='cosine', random_state=42).fit_transform(core_latent_sample) #change n_neighbors to emphasize a local or global set
dbscan = DBSCAN(eps=1.0, min_samples=10)
dbscan_labels = dbscan.fit_predict(umap_embedding)

In [ ]:
# PCA with DBSCAN
dim_reducer_pca = PCA(n_components=2)
pca_embedding = dim_reducer_pca.fit_transform(core_latent_sample)
dbscan_pca = DBSCAN(eps=0.5, min_samples=10)
dbscan_pca_labels = dbscan_pca.fit_predict(pca_embedding)

# Other embeddings (just to see)
tsne_reducer = TSNE(n_components=2, random_state=42)
tsne_embedding = tsne_reducer.fit_transform(core_latent_sample)

isomap_reducer = Isomap(n_components=2)
isomap_embedding = isomap_reducer.fit_transform(core_latent_sample)

spectral_embedder = SpectralEmbedding(n_components=2, random_state=42)
spectral_embedding = spectral_embedder.fit_transform(core_latent_sample)

In [ ]:
# Hybrid approachs (there is no difference between plotting with different attribute names using DBSCAN)
for attribute_name, attribute_values in attributes.items():
    attribute_sample = attribute_values[sample_indices]

    # UMAP with DBSCAN
    plot_latent_space(latent_sample, attribute_sample, attribute_name, 'UMAP with DBSCAN', umap_embedding, ['UMAP Dimension 1', 'UMAP Dimension 2'], blob_labels=dbscan_labels)

    # PCA with DBSCAN
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 'PCA with DBSCAN', pca_embedding, ['PCA Component 1', 'PCA Component 2'], blob_labels=dbscan_pca_labels)

In [ ]:
for attribute_name, attribute_values in attributes.items():
    attribute_sample = attribute_values[sample_indices]

    # PCA
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 'PCA', pca_embedding, ['PCA Component 1', 'PCA Component 2'])

    # t-SNE
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 't-SNE', tsne_embedding, ['t-SNE Dimension 1', 't-SNE Dimension 2'])

    # UMAP
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 'UMAP', umap_embedding, ['UMAP Dimension 1', 'UMAP Dimension 2'])

    # Isomap
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 'Isomap', isomap_embedding, ['Isomap Dimension 1', 'Isomap Dimension 2'])

    # Spectral Embedding
    plot_latent_space(core_latent_sample, attribute_sample, attribute_name, 'Spectral Embedding', spectral_embedding, ['Spectral Dimension 1', 'Spectral Dimension 2'])

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(latent_sample)

# K-Means
plot_latent_space(core_latent_sample, None, None, 'K-Means Clustering', latent_sample[:, :2], ['Latent Dimension 1', 'Latent Dimension 2'], blob_labels=kmeans_labels)

# DBSCAN
plot_latent_space(core_latent_sample, None, None, 'DBSCAN blobing', latent_sample[:, :2], ['Latent Dimension 1', 'Latent Dimension 2'], blob_labels=dbscan_labels)


In [ ]:
attribute_sample = attributes['eta'][sample_indices]

n_neighbors_values = range(10, 101, 10)  #from 10 to 100 in increments of 10
for n_neighbors in n_neighbors_values:

    umap_reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=0.2,
        n_components=2,
        metric='cosine',
        random_state=42
    )
    umap_embedding = umap_reducer.fit_transform(latent_sample)

    dbscan = DBSCAN(eps=1.0, min_samples=10)
    dbscan_labels = dbscan.fit_predict(umap_embedding)

    plot_latent_space(
        latent_sample=latent_sample,
        attribute_sample=attribute_sample,
        attribute_name='Eta',
        technique_name=f'UMAP with DBSCAN (n_neighbors={n_neighbors})',
        embedding=umap_embedding,
        component_labels=['UMAP Dimension 1', 'UMAP Dimension 2'],
        blob_labels=dbscan_labels
    )

In [ ]:
def generate_dynamic_title(eta_threshold, condition_type="less"):
    eta_str = f"eta {'<' if condition_type == 'less' else '>'} {eta_threshold}"
    return f'Filtered UMAP Embedding Highlighting All Wafer Types ({eta_str})'

eta = conditions_subset[:, 0]
wafertype_onehot = conditions_subset[:, 3:6]

eta_threshold = 0.5
eta_below_threshold_mask = eta < eta_threshold
eta_above_threshold_mask = eta >= eta_threshold

fig, axes = plt.subplots(2, 2, figsize=(28, 20))

# Original UMAP colored by eta
ax = axes[0, 0]
scatter = ax.scatter(
    umap_embedding[:, 0], umap_embedding[:, 1],
    c=eta, cmap='viridis', s=20, alpha=0.8, edgecolor='k'
)
ax.set_title('Original UMAP Embedding Colored by eta', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
cbar = plt.colorbar(scatter, ax=ax, label=r'$\eta$', shrink=0.8)
ax.grid(visible=True, linestyle='--', linewidth=0.5)

# Original UMAP colored by wafer type
ax = axes[0, 1]
wafer_colors = ['#0000FF', '#00FF00', '#FF0000']
for i, color in enumerate(wafer_colors):
    mask = wafertype_onehot[:, i] == 1
    ax.scatter(
        umap_embedding[mask, 0], umap_embedding[mask, 1],
        s=20, alpha=0.8, color=color, edgecolor='k', label=f'Wafer Type {i + 1}'
    )
ax.set_title('Original UMAP Embedding Colored by Wafer Type', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
ax.legend(title='Wafer Types', loc='best')
ax.grid(visible=True, linestyle='--', linewidth=0.5)

# Filtered UMAP for eta < threshold
ax = axes[1, 0]
legend_labels_added = set()  #avoid redundant legend entries
low_eta_colors = ['#00008B', '#00FFFF', '#7B68EE']

for label in np.unique(dbscan_labels):
    mask = dbscan_labels == label
    ax.scatter(
        umap_embedding[mask, 0], umap_embedding[mask, 1],
        s=20, alpha=0.1, color='gray', edgecolor='k'
    )

# Highlight filtered blobs
for i, color in enumerate(low_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_below_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.9, color=color, edgecolor='k',
        label=f'Wafer Type {i + 1}' if f'Wafer Type {i + 1}' not in legend_labels_added else None
    )
    legend_labels_added.add(f'Wafer Type {i + 1}')

title = generate_dynamic_title(eta_threshold=eta_threshold, condition_type="less")
ax.set_title(title, fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
ax.legend(title='Filtered Blobs', loc='best')
ax.grid(visible=True, linestyle='--', linewidth=0.5)

# Filtered UMAP for eta > threshold
ax = axes[1, 1]
legend_labels_added = set()  
high_eta_colors = ['#8B0000', '#FF6347', '#FF69B4']

for label in np.unique(dbscan_labels):
    mask = dbscan_labels == label
    ax.scatter(
        umap_embedding[mask, 0], umap_embedding[mask, 1],
        s=20, alpha=0.1, color='gray', edgecolor='k'
    )

for i, color in enumerate(high_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_above_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.9, color=color, edgecolor='k',
        label=f'Wafer Type {i + 1}' if f'Wafer Type {i + 1}' not in legend_labels_added else None
    )
    legend_labels_added.add(f'Wafer Type {i + 1}')

title = generate_dynamic_title(eta_threshold=eta_threshold, condition_type="greater")
ax.set_title(title, fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
ax.legend(title='Filtered Blobs', loc='best')
ax.grid(visible=True, linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
low_eta_colors = ['#1E90FF', '#00FFFF', '#7B68EE']  # Dodger blue, cyan, slate blue
high_eta_colors = ['#FF4500', '#A52A2A', '#FF69B4']  # Orange red, dark red, hot pink

# Mask definitions for low and high eta
eta_threshold = 0.5
eta_below_threshold_mask = eta < eta_threshold
eta_above_threshold_mask = eta >= eta_threshold

fig, ax = plt.subplots(figsize=(14, 10))

# Low eta
for i, color in enumerate(low_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_below_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.8, color=color, edgecolor='k'
    )

# High eta
for i, color in enumerate(high_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_above_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.8, color=color, edgecolor='k'
    )

# Custom legends
low_eta_legend = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=low_eta_colors[0],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 1'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=low_eta_colors[1],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 2'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=low_eta_colors[2],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 3')
]

high_eta_legend = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=high_eta_colors[0],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 1'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=high_eta_colors[1],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 2'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=high_eta_colors[2],
           markeredgecolor='black', markeredgewidth=1.5, markersize=10, label='Wafer-type 3')
]

legend_low = ax.legend(handles=low_eta_legend, title=r'   Low${\ \eta}$', loc='lower right', fontsize=15, frameon=True, title_fontsize=16)
ax.add_artist(legend_low)

legend_high = ax.legend(handles=high_eta_legend, title=r'   High${\ \eta}$', loc='upper right', fontsize=15, frameon=True, title_fontsize=16)
ax.add_artist(legend_high)

ax.set_title('', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=20)
ax.set_ylabel('UMAP Dimension 2', fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
def plot_density_distribution(embedding, dbscan_labels):
    plt.figure(figsize=(10, 6))
    x, y = embedding[:, 0], embedding[:, 1]
    kde = gaussian_kde([x, y])
    density = kde([x, y])

    # Determine points from DBSCAN labels
    core_samples_mask = dbscan_labels != -1
    outliers_mask = dbscan_labels == -1

    plt.scatter(x[outliers_mask], y[outliers_mask], c='red', s=20, label='Outliers', alpha=0.8, edgecolor='k')
    plt.scatter(x[core_samples_mask], y[core_samples_mask], c=density[core_samples_mask], s=20, cmap='viridis', label='Core Points', alpha=0.8, edgecolor='k')

    plt.title('Density Distribution of UMAP Embedding', fontsize=16)
    plt.colorbar(label='Density', shrink=0.8)
    plt.xlabel('UMAP Dimension 1', fontsize=14)
    plt.ylabel('UMAP Dimension 2', fontsize=14)
    plt.legend()
    plt.grid(visible=True, linestyle='--', linewidth=0.5)
    plt.show()

plot_density_distribution(umap_embedding, dbscan_labels)

In [ ]:
def plot_highlight_blob(latent_sample, blob_labels, blob_to_highlight, color_map):
    plt.figure(figsize=(14, 10))

    unique_labels = np.unique(blob_labels)
    for label in unique_labels:
        if label == blob_to_highlight:
            # Highlight selected blob
            mask = blob_labels == label
            plt.scatter(
                latent_sample[mask, 0], latent_sample[mask, 1],
                s=20, alpha=0.9, color=color_map.get(label, 'blue'), edgecolor='k',
                label=f'Blob {label + 1}'
            )
            x_text = np.mean(latent_sample[mask, 0])
            y_text = np.mean(latent_sample[mask, 1])
            plt.text(
                x_text, y_text, f'Blob {label + 1}', fontsize=12, weight='bold',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
            )
        else:
            # Gray out other blobs
            mask = blob_labels == label
            plt.scatter(
                latent_sample[mask, 0], latent_sample[mask, 1],
                s=20, alpha=0.1, color='gray', edgecolor='k'
            )

    if -1 in unique_labels:
        mask = blob_labels == -1
        plt.scatter(
            latent_sample[mask, 0], latent_sample[mask, 1],
            s=20, alpha=0.9, color='#FF0000', edgecolor='k',
            label='Outlier'
        )

    plt.title(f'UMAP with DBSCAN Projection of Latent Space with Highlighted Blob {blob_to_highlight + 1}', fontsize=16)
    plt.xlabel('UMAP Dimension 1', fontsize=14)
    plt.ylabel('UMAP Dimension 2', fontsize=14)
    plt.legend(title='Blob Labels', loc='best')
    plt.grid(visible=True, linestyle='--', linewidth=0.5)
    plt.show()

def plot_physical_quantities(blob_labels, conditions, blob_index):
    blob_mask = blob_labels == blob_index
    blob_conditions = conditions[blob_mask]

    eta = blob_conditions[:, 0]
    wafer_v = blob_conditions[:, 1]
    wafer_u = blob_conditions[:, 2]
    sum_CALQ = blob_conditions[:, -2]
    layers = blob_conditions[:, -1]

    wafertype_1 = blob_conditions[:, 3]
    wafertype_2 = blob_conditions[:, 4]
    wafertype_3 = blob_conditions[:, 5]

    # Subplots for different physical quantities
    fig, axes = plt.subplots(3, 3, figsize=(20, 18))

    # Eta
    ax = axes[0, 0]
    ax.hist(eta, bins=30, color='blue', alpha=0.7)
    ax.set_title(f'Distribution of Eta for Blob {blob_index + 1}')
    ax.set_xlabel('Eta')

    # Wafer_v
    ax = axes[0, 1]
    ax.hist(wafer_v, bins=30, color='green', alpha=0.7)
    ax.set_title(f'Distribution of Wafer V for Blob {blob_index + 1}')
    ax.set_xlabel('Wafer V')

    # Wafer_u
    ax = axes[0, 2]
    ax.hist(wafer_u, bins=30, color='purple', alpha=0.7)
    ax.set_title(f'Distribution of Wafer U for Blob {blob_index + 1}')
    ax.set_xlabel('Wafer U')

    # Sum_CALQ
    ax = axes[1, 0]
    ax.hist(sum_CALQ, bins=30, color='red', alpha=0.7)
    ax.set_title(f'Distribution of Sum CALQ for Blob {blob_index + 1}')
    ax.set_xlabel('Sum CALQ')

    # Layers
    ax = axes[1, 1]
    ax.hist(layers, bins=30, color='orange', alpha=0.7)
    ax.set_title(f'Distribution of Layers for Blob {blob_index + 1}')
    ax.set_xlabel('Layers')

    # Wafertype_1
    ax = axes[2, 0]
    ax.hist(wafertype_1, bins=30, color='black', alpha=0.7)
    ax.set_title(f'Distribution of Wafer Type 1 for Blob {blob_index + 1}')
    ax.set_xlabel('Wafer Type 1')

    # Wafertype_2
    ax = axes[2, 1]
    ax.hist(wafertype_2, bins=30, color='gray', alpha=0.7)
    ax.set_title(f'Distribution of Wafer Type 2 for Blob {blob_index + 1}')
    ax.set_xlabel('Wafer Type 2')

    # Wafertype_3
    ax = axes[2, 2]
    ax.hist(wafertype_3, bins=30, color='#800080', alpha=0.7)  # Purple
    ax.set_title(f'Distribution of Wafer Type 3 for Blob {blob_index + 1}')
    ax.set_xlabel('Wafer Type 3')

    plt.tight_layout()
    plt.show()

color_map = {
    0: 'blue',
    1: 'green',
    2: 'purple',
    3: 'red',
    4: 'orange',
    5: 'black',
    6: 'gray',
    7: 'pink',
    8: 'brown',
    9: 'yellow',
    10: 'cyan'
}


blob_to_highlight = 1  # Blob to highlight
plot_highlight_blob(umap_embedding, dbscan_labels, blob_to_highlight, color_map)

plot_physical_quantities(dbscan_labels, conditions_subset, blob_to_highlight)

In [ ]:
def get_color_map(blob_labels):
    unique_labels = np.unique(blob_labels)
    colors = sns.color_palette('husl', len(unique_labels))
    color_map = {label: colors[i] for i, label in enumerate(unique_labels)}
    return color_map

# Highlight two blobs
def plot_highlight_two_blobs(latent_sample, blob_labels, blob_1, blob_2, color_map):
    plt.figure(figsize=(14, 10))
    unique_labels = np.unique(blob_labels)

    for label in unique_labels:
        if label == blob_1:
            # First blob
            mask = blob_labels == label
            plt.scatter(latent_sample[mask, 0], latent_sample[mask, 1], s=20, alpha=0.9, label=f'Blob {label + 1}',
                        edgecolor='k', color=color_map[label])
            x_text = np.mean(latent_sample[mask, 0])
            y_text = np.mean(latent_sample[mask, 1])
            plt.text(x_text, y_text, f'Blob {label + 1}', fontsize=12, weight='bold',
                     bbox=dict(facecolor='white', alpha=0.7, edgecolor='black'))
        elif label == blob_2:
            # Second blob
            mask = blob_labels == label
            plt.scatter(latent_sample[mask, 0], latent_sample[mask, 1], s=20, alpha=0.9, label=f'Blob {label + 1}',
                        edgecolor='k', color=color_map[label])
            x_text = np.mean(latent_sample[mask, 0])
            y_text = np.mean(latent_sample[mask, 1])
            plt.text(x_text, y_text, f'Blob {label + 1}', fontsize=12, weight='bold',
                     bbox=dict(facecolor='white', alpha=0.7, edgecolor='black'))
        else:
            mask = blob_labels == label
            plt.scatter(latent_sample[mask, 0], latent_sample[mask, 1], s=20, alpha=0.1, color='gray', edgecolor='k')

    if -1 in unique_labels:
        mask = blob_labels == -1
        plt.scatter(latent_sample[mask, 0], latent_sample[mask, 1], s=20, alpha=0.9, label='Outlier',
                    color='#FF0000', edgecolor='k')

    plt.title(f'UMAP with DBSCAN Projection of Latent Space with Highlighted Blobs {blob_1 + 1} and {blob_2 + 1}', fontsize=16)
    plt.xlabel('UMAP Dimension 1', fontsize=14)
    plt.ylabel('UMAP Dimension 2', fontsize=14)
    plt.legend(title='Blob Labels', loc='best')
    plt.grid(visible=True, linestyle='--', linewidth=0.5)
    plt.show()

# Overlay condition distributions for the two highlighted blobs
def plot_overlay_distribution(conditions, blob_labels, blob_1, blob_2, attribute_index, attribute_name, color_map):
    attribute_1 = conditions[blob_labels == blob_1, attribute_index]
    attribute_2 = conditions[blob_labels == blob_2, attribute_index]

    plt.figure(figsize=(10, 6))

    color_1 = color_map[blob_1]
    color_2 = color_map[blob_2]

    sns.kdeplot(attribute_1, color=color_1, label=f'Blob {blob_1 + 1}', linestyle='-', linewidth=2)
    sns.kdeplot(attribute_2, color=color_2, label=f'Blob {blob_2 + 1}', linestyle='-', linewidth=2)

    plt.title(f'Overlayed {attribute_name} Distributions for Blobs', fontsize=16)
    plt.xlabel(attribute_name, fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend(title='Blob Labels', loc='best')
    plt.grid(visible=True, linestyle='--', linewidth=0.5)
    plt.show()

blob_1_to_highlight = 2
blob_2_to_highlight = 0

color_map = get_color_map(dbscan_labels)

plot_highlight_two_blobs(umap_embedding, dbscan_labels, blob_1_to_highlight, blob_2_to_highlight, color_map)

plot_overlay_distribution(conditions_subset, dbscan_labels, blob_1_to_highlight, blob_2_to_highlight, 0, 'Eta', color_map)

In [ ]:
valid_data_mask = dbscan_labels != -1
X = conditions_subset[valid_data_mask]
y = dbscan_labels[valid_data_mask]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Forest Classifier to predict clustering Labels
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

feature_names = [
    'eta', 'wafer_v', 'wafer_u', 'wafertype_1', 'wafertype_2', 'wafertype_3',
    'sum_CALQ', 'layers_normalized'
]

X_train_df = pd.DataFrame(X_train, columns=feature_names)
explainer = shap.TreeExplainer(rf_classifier)

# Calculate SHAP values for the training set
shap_values = explainer.shap_values(X_train)

if isinstance(shap_values, list):
    shap_values_class = shap_values[1]
else:
    shap_values_class = shap_values

shap.summary_plot(shap_values_class, X_train_df)

# Permutation importance for data
perm_importance = permutation_importance(rf_classifier, X_test, y_test, n_repeats=30, random_state=42)

perm_sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(10, 6))
plt.barh(range(len(perm_sorted_idx)), perm_importance.importances_mean[perm_sorted_idx], align='center')
plt.yticks(range(len(perm_sorted_idx)), [feature_names[i] for i in perm_sorted_idx])
plt.xlabel("Permutation Importance")
plt.title("Feature Importance via Permutation Importance")
plt.show()

In [ ]:
import numba

# Adjust weights to enforce more emphasis on certain attributes
eta_weight = 5.0
wafer_type_weight = 1.5

@numba.njit()
def custom_distance(x, y):
    eta_diff = eta_weight * abs(x[0] - y[0])
    wafer_type_diff = wafer_type_weight * abs(x[1] - y[1])
    other_features_diff = np.linalg.norm(x[2:] - y[2:])
    return eta_diff + wafer_type_diff + other_features_diff

umap_reducer_custom = umap.UMAP(n_neighbors=100, min_dist=0.1, metric=custom_distance, random_state=42)
umap_embedding_custom = umap_reducer_custom.fit_transform(latent_sample)

dbscan_custom = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels_custom = dbscan_custom.fit_predict(umap_embedding)

In [ ]:
for attribute_name, attribute_values in attributes.items():
    attribute_sample = attribute_values[sample_indices]

    # Custom distance metric
    plot_latent_space(umap_embedding_custom, attribute_sample, attribute_name, 'Custom UMAP with DBSCAN', umap_embedding_custom, ['UMAP Dimension 1', 'UMAP Dimension 2'], blob_labels=None)

In [ ]:
def generate_dynamic_title(wafer_type_mask, eta_mask, wafer_type_index, eta_threshold, condition_type="less"):
    wafer_type_str = f"wafer type {wafer_type_index + 1}"
    eta_str = f"eta {'<' if condition_type == 'less' else '>'} {eta_threshold}"
    return f'Filtered UMAP Embedding with Highlighted Blobs ({wafer_type_str} & {eta_str})'

eta = conditions_subset[:, 0]
wafertype_onehot = conditions_subset[:, 3:6]
sumCALQ = conditions_subset[:, 6]
layers_normalized = conditions_subset[:, 7]

# Filter mask for wafer type and eta
wafer_type_mask = wafertype_onehot[:, 0] == 1
eta_below_threshold_mask = eta < 0.5
combined_mask = wafer_type_mask & eta_below_threshold_mask

highlighted_embedding = umap_embedding[combined_mask]
highlighted_labels = dbscan_labels[combined_mask]

fig, axes = plt.subplots(1, 2, figsize=(28, 10))

ax = axes[0]
scatter = ax.scatter(
    umap_embedding[:, 0], umap_embedding[:, 1],
    c=eta, cmap='viridis', s=20, alpha=0.8, edgecolor='k'
)
ax.set_title('Original UMAP Embedding Colored by eta', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
cbar = plt.colorbar(scatter, ax=ax, label='eta', shrink=0.8)
ax.grid(visible=True, linestyle='--', linewidth=0.5)

# Filtered UMAP with only selected clusters highlighted
ax = axes[1]
for label in np.unique(dbscan_labels):
    if label == -1:
        mask = dbscan_labels == label
        ax.scatter(
            umap_embedding[mask, 0], umap_embedding[mask, 1],
            s=20, alpha=0.2, color='#FF0000', edgecolor='k', label='Outlier' if label == -1 else None
        )
    elif label in highlighted_labels:
        mask = dbscan_labels == label
        ax.scatter(
            umap_embedding[mask, 0], umap_embedding[mask, 1],
            s=20, alpha=0.9, color=color_map[label], edgecolor='k', label=f'Filtered Blob {label + 1}'
        )
    else:
        mask = dbscan_labels == label
        ax.scatter(
            umap_embedding[mask, 0], umap_embedding[mask, 1],
            s=20, alpha=0.1, color='gray', edgecolor='k'
        )

title = generate_dynamic_title(
    wafer_type_mask,
    eta_below_threshold_mask,
    wafer_type_index=2,  #adjust based off selection above
    eta_threshold=0.5,
    condition_type="less"
)

ax.set_title(title, fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
ax.legend(title='Filtered Blobs', loc='best')
ax.grid(visible=True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
eta = conditions_subset[:, 0]
wafertype_onehot = conditions_subset[:, 3:6]

eta_threshold = 0.5
eta_below_threshold_mask = eta < eta_threshold
eta_above_threshold_mask = eta >= eta_threshold

low_eta_colors = ['#1E90FF', '#00FFFF', '#7B68EE']  #eta < 0.5
high_eta_colors = ['#FF4500', '#A52A2A', '#FF69B4']  #eta >= 0.5

fig, ax = plt.subplots(figsize=(14, 10))
for label in np.unique(dbscan_labels):
    mask = dbscan_labels == label
    ax.scatter(
        umap_embedding[mask, 0], umap_embedding[mask, 1],
        s=20, alpha=0.1, color='gray', edgecolor='k'
    )

# Plot points with eta < 0.5
for i, color in enumerate(low_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_below_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.9, color=color, edgecolor='k',
        label=f'Wafer Type {i + 1} (eta < 0.5)'
    )

# Plot points with eta >= 0.5
for i, color in enumerate(high_eta_colors):
    wafer_type_mask = wafertype_onehot[:, i] == 1
    combined_mask = eta_above_threshold_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_mask, 0], umap_embedding[combined_mask, 1],
        s=20, alpha=0.9, color=color, edgecolor='k',
        label=f'Wafer Type {i + 1} (eta >= 0.5)'
    )

ax.set_title('Combined UMAP Embedding Highlighting All Wafer Types by Eta Threshold', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
ax.legend(title='Wafer Types with Eta', loc='best')
ax.grid(visible=True, linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
eta = conditions_subset[:, 0]
wafertype_onehot = conditions_subset[:, 3:6]

eta_threshold = 0.5
low_eta_mask = eta < eta_threshold
high_eta_mask = eta >= eta_threshold

fig, ax = plt.subplots(figsize=(10, 8))

x_min, x_max = -10, 10
y_min, y_max = -10, 10

low_eta_color = 'black'  #eta < 0.5
high_eta_color = 'white'  ##eta >= 0.5

# Outline colors for wafer types
wafer_outlines = ['blue', 'green', 'red']

legend_labels_added = set()

# Plot the filtered UMAP points with distinct outlines for wafer types and fill colors for eta levels
for wafer_type_idx in range(wafertype_onehot.shape[1]):
    wafer_type_mask = wafertype_onehot[:, wafer_type_idx] == 1
    outline_color = wafer_outlines[wafer_type_idx]

    # Low eta points for specific wafer type
    combined_low_eta_mask = low_eta_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_low_eta_mask, 0], umap_embedding[combined_low_eta_mask, 1],
        s=60, alpha=0.9, color=low_eta_color, edgecolor=outline_color,
        linewidth=1.5,
        label=f'Wafer Type {wafer_type_idx + 1}' if f'Wafer Type {wafer_type_idx + 1}' not in legend_labels_added else None
    )
    legend_labels_added.add(f'Wafer Type {wafer_type_idx + 1}')

    # High eta points for specfic wafer type
    combined_high_eta_mask = high_eta_mask & wafer_type_mask
    ax.scatter(
        umap_embedding[combined_high_eta_mask, 0], umap_embedding[combined_high_eta_mask, 1],
        s=60, alpha=0.9, color=high_eta_color, edgecolor=outline_color,
        linewidth=1.5
    )

wafer_legend_handles = [
    mlines.Line2D([], [], color='none', marker='o', linestyle='None', markersize=10, markeredgecolor='blue', label='Wafer Type 1'),
    mlines.Line2D([], [], color='none', marker='o', linestyle='None', markersize=10, markeredgecolor='green', label='Wafer Type 2'),
    mlines.Line2D([], [], color='none', marker='o', linestyle='None', markersize=10, markeredgecolor='red', label='Wafer Type 3')
]

separator_line = mlines.Line2D([], [], color='gray', linestyle='-', linewidth=1, label=' ')

eta_legend_handles = [
    mlines.Line2D([], [], color='black', marker='o', markersize=10, linestyle='None', markerfacecolor='black', label=r'Low $\eta$'),
    mlines.Line2D([], [], color='black', marker='o', markersize=10, linestyle='None', markerfacecolor='white', label=r'High $\eta$')
]

handles = wafer_legend_handles + [separator_line] + eta_legend_handles
legend = ax.legend(handles=handles, title='', loc='lower left', frameon=True)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots(figsize=(10, 8))

custom_cmap = LinearSegmentedColormap.from_list('blue_red', ['blue', 'red'])

scatter = ax.scatter(
    umap_embedding[:, 0], umap_embedding[:, 1],
    c=conditions_subset[:, 0], cmap=custom_cmap, s=20, alpha=0.8, edgecolor='k'
)
ax.set_title('', fontsize=16)
ax.set_xlabel('UMAP Dimension 1', fontsize=14)
ax.set_ylabel('UMAP Dimension 2', fontsize=14)
cbar = plt.colorbar(scatter, ax=ax, label='$\eta$')

# Ellipses for each wafer type
ellipse_params = [
    {'xy': (8, 21), 'width': 29, 'height': 7, 'angle': 5, 'edgecolor': 'black', 'label': 'Wafer-type 1', 'linestyle': '-'},
    {'xy': (5, 1.85), 'width': 40, 'height': 7, 'angle': -24, 'edgecolor': 'black', 'label': 'Wafer-type 2', 'linestyle': (0, (3, 3))},
    {'xy': (2.5, -11), 'width': 41, 'height': 10, 'angle': -36, 'edgecolor': 'black', 'label': 'Wafer-type 3', 'linestyle': (0, (1, 2))}
]

for params in ellipse_params:
    ellipse = Ellipse(**params, facecolor='none', linewidth=1)
    ax.add_patch(ellipse)

legend_handles = [
    Ellipse((0, 0), 1, 1, edgecolor='black', facecolor='none', linewidth=1, linestyle='-', label='Wafer-type 1'),
    Ellipse((0, 0), 1, 1, edgecolor='black', facecolor='none', linewidth=1, linestyle=(0, (3, 3)), label='Wafer-type 2'),
    Ellipse((0, 0), 1, 1, edgecolor='black', facecolor='none', linewidth=1, linestyle=(0, (1, 2)), label='Wafer-type 3')
]
ax.legend(handles=legend_handles, loc='best', title='')

plt.tight_layout()
plt.show()

In [ ]:
def plot_density_contours(embedding, labels, title="Density Contours in the Latent Space"):
    plt.figure(figsize=(10, 8))
    sns.kdeplot(
        x=embedding[:, 0], y=embedding[:, 1], hue=labels, fill=True,
        palette="viridis", alpha=0.5, thresh=0.1, levels=10
    )
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.title(title)
    plt.show()

plot_density_contours(umap_embedding, dbscan_labels)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_3d_latent_space(embedding, labels, color_map, title="3D Latent Space Projection"):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(
        embedding[:, 0], embedding[:, 1], embedding[:, 2], c=labels,
        cmap=color_map, s=20, alpha=0.7, edgecolor='k'
    )
    plt.colorbar(scatter)
    ax.set_xlabel('Latent Dimension 1')
    ax.set_ylabel('Latent Dimension 2')
    ax.set_zlabel('Latent Dimension 3')
    ax.set_title(title)
    plt.show()

umap_3d_embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=3, metric='euclidean').fit_transform(latent_sample)

plot_3d_latent_space(umap_3d_embedding, labels=dbscan_labels, color_map="viridis")

In [ ]:
from scipy.spatial.distance import pdist, squareform

def plot_pairwise_distance_heatmap(embedding, labels, title="Pairwise Distance Heatmap"):
    unique_labels = np.unique(labels)
    centroids = [embedding[labels == label].mean(axis=0) for label in unique_labels]
    distance_matrix = squareform(pdist(centroids))

    plt.figure(figsize=(10, 8))
    sns.heatmap(distance_matrix, annot=True, xticklabels=unique_labels, yticklabels=unique_labels, cmap="Blues")
    plt.title(title)
    plt.show()

plot_pairwise_distance_heatmap(umap_embedding, dbscan_labels)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.spatial.distance import pdist

# Silhouette Score
silhouette = silhouette_score(umap_embedding, dbscan_labels)
print(f"Silhouette Score: {silhouette}")

# Davies-Bouldin Index
db_index = davies_bouldin_score(umap_embedding, dbscan_labels)
print(f"Davies-Bouldin Index: {db_index}")

from sklearn.metrics import silhouette_samples

silhouette_values = silhouette_samples(umap_embedding, dbscan_labels)

# Normalize scores
norm_silhouette = (silhouette_values - silhouette_values.min()) / (silhouette_values.max() - silhouette_values.min())

plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    umap_embedding[:, 0], umap_embedding[:, 1],
    c=norm_silhouette, cmap='coolwarm', s=20, alpha=0.8, edgecolor='k'
)
plt.colorbar(scatter, label="Normalized Silhouette Score")
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.title('')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_samples

# UMAP with euclidean distance metric
umap_euclidean = umap.UMAP(n_neighbors=15, min_dist=0.2, n_components=2, metric='euclidean', random_state=42)
umap_embedding_euclidean = umap_euclidean.fit_transform(latent_sample)

silhouette_values_euclidean = silhouette_samples(umap_embedding_euclidean, dbscan_labels)

norm_silhouette_euclidean = (silhouette_values_euclidean - silhouette_values_euclidean.min()) / (silhouette_values_euclidean.max() - silhouette_values_euclidean.min())

plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    umap_embedding_euclidean[:, 0], umap_embedding_euclidean[:, 1],
    c=norm_silhouette_euclidean, cmap='coolwarm', s=20, alpha=0.8, edgecolor='k'
)
plt.colorbar(scatter, label="Normalized Silhouette Score")
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.title('UMAP Embedding with Euclidean Metric')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
metrics = ['cosine', 'euclidean', 'manhattan', 'chebyshev', 'correlation']
umap_embeddings = {}

sample_indices = np.random.choice(len(latent_sample), num_samples, replace=False)
latent_sample_subset = latent_sample[sample_indices]
attribute_sample = attributes['eta'][sample_indices]

for metric in metrics:

    umap_reducer = umap.UMAP(n_neighbors=100, min_dist=0.1, n_components=2, metric=metric, random_state=42)
    umap_embedding = umap_reducer.fit_transform(latent_sample_subset)
    umap_embeddings[metric] = umap_embedding

    dbscan = DBSCAN(eps=0.5, min_samples=10)
    dbscan_labels = dbscan.fit_predict(umap_embedding)

    plot_latent_space(
        latent_sample=latent_sample,
        attribute_sample=attribute_sample,
        attribute_name='Eta',
        technique_name=f'UMAP with {metric.capitalize()} Metric',
        embedding=umap_embedding,
        component_labels=['UMAP Dimension 1', 'UMAP Dimension 2'],
        blob_labels=dbscan_labels
    )

In [ ]:
# eLinks list
eLinks = [2, 3, 4, 5]
num_samples = 5000

batch_size = 128
inputs, conditions = load_root_files(file_limit=-1)
inputs = np.expand_dims(inputs, axis=-1)

for eLink in eLinks:
    encoder = EncoderModelBuilder.build_encoder_model()

    weights_file = f'encoder_vanilla_AE_eLink{eLink}.hdf5'
    encoder.load_weights(weights_file)
    print(f"Loaded weights for eLink {eLink} from {weights_file}")

    latent_representations = encoder.predict([inputs, conditions], batch_size=batch_size)

    sample_indices = np.random.choice(len(latent_representations), num_samples, replace=False)
    latent_sample = latent_representations[sample_indices]
    attribute_sample = attributes['eta'][sample_indices]

    umap_reducer = umap.UMAP(n_neighbors=55, min_dist=0.2, n_components=2, metric='cosine', random_state=42)
    umap_embedding = umap_reducer.fit_transform(latent_sample)

    dbscan = DBSCAN(eps=1.0, min_samples=10)
    dbscan_labels = dbscan.fit_predict(umap_embedding)

    plot_latent_space(
        latent_sample=latent_sample,
        attribute_sample=attribute_sample,
        attribute_name='Eta',
        technique_name=f'UMAP with DBSCAN for eLink {eLink}',
        embedding=umap_embedding,
        component_labels=['UMAP Dimension 1', 'UMAP Dimension 2'],
        blob_labels=dbscan_labels
    )

In [ ]:
# Load ROOT Files to process a SINGLE EVENT
def load_single_event(file_limit=-1, selected_eLinks=-1):
    files = glob.glob('*.root')
    if file_limit > 0:
        files = files[:file_limit]

    all_inputs, all_conditions = [], []
    tree_name = 'FloatingpointThreshold0DummyHistomaxDummynTuple/HGCalTriggerNtuple'

    for file_index, file in enumerate(files):
        print(f"Processing file {file_index + 1}/{len(files)}: {file}")
        try:
            with uproot.open(file) as root_file:
                if tree_name not in root_file:
                    raise ValueError(f"Tree '{tree_name}' not found in file '{file}'")
                tree = root_file[tree_name]

                branches = [
                    "gen_pt", "wafer_layer", "wafer_eta", "wafer_waferv",
                    "wafer_waferu", "wafer_wafertype"
                ]
                branches.extend([f"wafer_CALQ{j}" for j in range(64)])
                branches.extend([f"wafer_AEin{j}" for j in range(64)])

                data = tree.arrays(branches, library="ak")
                mask = (ak.mean(data["gen_pt"], axis=1) >= 0) & (ak.mean(data["gen_pt"], axis=1) <= 100000)
                # Select the first event
                data = data[:1]

                print(f"Number of events processed: {len(data['gen_pt'])}")

                layers = ak.to_numpy(ak.flatten(data["wafer_layer"]))
                eta = ak.to_numpy(ak.flatten(data["wafer_eta"])) / 3.1
                wafer_v = ak.to_numpy(ak.flatten(data["wafer_waferv"])) / 12
                wafer_u = ak.to_numpy(ak.flatten(data["wafer_waferu"])) / 12
                wafer_type = ak.to_numpy(ak.flatten(data["wafer_wafertype"])).astype(int)
                one_hot_wafertype = np.eye(np.max(wafer_type) + 1)[wafer_type]

                sum_CALQ = np.sum([ak.to_numpy(ak.flatten(data[f"wafer_CALQ{j}"])) for j in range(64)], axis=0)
                sum_CALQ = np.log(sum_CALQ + 1)

                inputs = np.stack([ak.to_numpy(ak.flatten(data[f"wafer_AEin{j}"])) for j in range(64)], axis=-1)
                inputs = np.reshape(inputs, (-1, 8, 8))

                # Verify individual TCs
                print("Checking individual trigger cells:")
                for j in range(64):
                    print(f"wafer_AEin{j}: {data[f'wafer_AEin{j}'][0]}")

                # Verify grid index mapping
                print("\nReshaped 8x8 Grid for First Wafer:")
                print(inputs[0].reshape(8, 8))

                # Plot individual trigger cell values
                for j in range(64):
                  wafer_grid = np.zeros((8, 8))

                  # Set the corresponding trigger cell position to its value
                  wafer_grid[j // 8, j % 8] = inputs[0, j // 8, j % 8]

                  plt.figure(figsize=(6, 6))
                  plt.imshow(wafer_grid, cmap='viridis', interpolation='none', vmin=0, vmax=1)
                  plt.title(f"Trigger Cell {j} in Wafer")
                  plt.colorbar(label='ADC Value')
                  plt.show()

                selection_mask = {
                    5: (layers <= 11) & (layers >= 5),
                    4: (layers == 7) | (layers == 11),
                    3: (layers == 13),
                    2: (layers < 7) | (layers > 13),
                    -1: (layers > 0)
                }[selected_eLinks]

                print(f"Selected eLinks: {selected_eLinks}")

                print("Applying the following selection mask:")
                if selected_eLinks == 5:
                    print("Mask: Layers between 5 and 11 (inclusive).")
                elif selected_eLinks == 4:
                    print("Mask: Layers 7 or 11.")
                elif selected_eLinks == 3:
                    print("Mask: Layer 13.")
                elif selected_eLinks == 2:
                    print("Mask: Layers outside 7 to 13 (i.e., less than 7 or greater than 13).")
                elif selected_eLinks == -1:
                    print("Mask: No filtering on layers (all layers > 0).")

                inputs = inputs[selection_mask]
                eta = eta[selection_mask]
                wafer_v = wafer_v[selection_mask]
                wafer_u = wafer_u[selection_mask]
                one_hot_wafertype = one_hot_wafertype[selection_mask]
                sum_CALQ = sum_CALQ[selection_mask]
                layers_normalized = (layers[selection_mask] - 1) / 46

                print("Unique layers after selection:", np.unique(layers[selection_mask]))
                print(f"Number of wafers after selection: {inputs.shape[0]}")

                conditions = np.hstack([
                    eta[:, np.newaxis], wafer_v[:, np.newaxis], wafer_u[:, np.newaxis],
                    one_hot_wafertype, sum_CALQ[:, np.newaxis], layers_normalized[:, np.newaxis]
                ])

                all_inputs.append(inputs)
                all_conditions.append(conditions)

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    if not all_inputs:
        raise ValueError("No data was loaded. Please check your ROOT files and tree names.")

    return np.concatenate(all_inputs), np.concatenate(all_conditions)

batch_size = 128

inputs, conditions = load_single_event(file_limit=-1)
print(inputs.shape)
print(conditions.shape)

inputs = np.expand_dims(inputs, axis=-1)
latent_representations = encoder.predict([inputs, conditions], batch_size)

In [ ]:
# Check on the ntuple
with uproot.open("ntuple_400.root") as root_file:
    tree = root_file["FloatingpointThreshold0DummyHistomaxDummynTuple/HGCalTriggerNtuple"]
    print(f"Total entries in the TTree: {tree.num_entries}")
    gen_pt = tree["gen_pt"].array(library="ak")
    print(ak.mean(gen_pt, axis=1))

    branch_names = tree.keys()
    print("Available branches:")
    for branch in branch_names:
        print(branch)

gen_pt_flat = ak.flatten(gen_pt)

plt.figure(figsize=(10, 6))
plt.hist(gen_pt_flat, bins=100, range=(0, 100), alpha=0.7, edgecolor='black')
plt.title("Distribution of gen_pt")
plt.xlabel("gen_pt")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

custom_cmap = LinearSegmentedColormap.from_list("green_red", ["green", "red"])

print("Shape of latent representations:", latent_representations.shape)

condition_names = [
    "Eta",                 # Index 0
    "Wafer V",             # Index 1
    "Wafer U",             # Index 2
    "One-Hot Wafertype 0", # Index 3
    "One-Hot Wafertype 1", # Index 4
    "One-Hot Wafertype 2", # Index 5
    "SumCALQ",             # Index 6
    "Layer Normalized"     # Index 7
]

# Each condition's unique values and range
for i, name in enumerate(condition_names):
    unique_values = np.unique(conditions[:, i])
    print(f"\nCondition {i} ({name}):")
    print(f"Unique Values: {unique_values}")
    print(f"Min: {np.min(conditions[:, i])}, Max: {np.max(conditions[:, i])}")


condition_indices = {name: idx for idx, name in enumerate(condition_names)}
print("\nCondition indices mapping:")
print(condition_indices)


colors = [
    'skyblue', 'orange', 'green', 'red', 'purple',
    'brown', 'pink', 'cyan'
]


plt.figure(figsize=(15, 20))

for i, name in enumerate(condition_names):
    plt.subplot(4, 2, i + 1)
    plt.hist(conditions[:, i], bins=50, color=colors[i], edgecolor='black', alpha=0.7)
    plt.title(f"{name} Distribution")
    plt.xlabel(name)
    plt.ylabel("Frequency")
    plt.grid(True)

plt.tight_layout()
plt.show()

sumCALQ_min, sumCALQ_max = 4, 6               # Threshold for SumCALQ
wafertype_onehot_value = [0, 1, 0]            # One-hot vector for Wafertype
eta_threshold = 0.5                           # Threshold for Eta
wafer_v_value = 1                             # Wafer V
wafer_u_value = 1                             # Wafer U
layer_min, layer_max = 0.2, 0.8               # Layer Normalized


# Boolean mask for filtering
mask = (
    np.all(conditions[:, condition_indices["One-Hot Wafertype 0"]:condition_indices["One-Hot Wafertype 2"] + 1] == wafertype_onehot_value, axis=1)
    #(conditions[:, condition_indices["Eta"]] > eta_threshold)
    #(conditions[:, condition_indices["SumCALQ"]] >= sumCALQ_min) &
    #(conditions[:, condition_indices["SumCALQ"]] <= sumCALQ_max)
)

print("\nApplying the filtering mask...")
print(f"Number of entries before filtering: {inputs.shape[0]}")
filtered_inputs = inputs[mask]
filtered_conditions = conditions[mask]
print(f"Number of entries after filtering: {filtered_inputs.shape[0]}")

plt.figure(figsize=(15, 20))

for i, name in enumerate(condition_names):
    plt.subplot(4, 2, i + 1)
    plt.hist(filtered_conditions[:, i], bins=50, color=colors[i], edgecolor='black', alpha=0.7)
    plt.title(f"{name} Distribution (Filtered)")
    plt.xlabel(name)
    plt.ylabel("Frequency")
    plt.grid(True)

plt.tight_layout()
plt.show()

# Number of wafers equals the number of latent points
num_wafers = inputs.shape[0]
num_latent_points = latent_representations.shape[0]

if num_wafers == num_latent_points:
    print("Each wafer module corresponds to one latent space point.")
else:
    print("Mismatch: The number of wafers does not match the number of latent space points.")



if filtered_inputs.size == 0 or filtered_conditions.size == 0:
    print("No data available after applying the filter. Adjust the filter conditions.")
else:
    print("\nGenerating latent representations...")
    filtered_latent_representations = encoder.predict([filtered_inputs, filtered_conditions], batch_size)
    print(f"Filtered Latent Representations Shape: {filtered_latent_representations.shape}")

    # Split the combined 24D output into 16D latent space and 8D conditions
    latent_space_16d = filtered_latent_representations[:, :16]


    sample_size = 5000
    if latent_space_16d.shape[0] > sample_size:
        idx = np.random.choice(latent_space_16d.shape[0], sample_size, replace=False)
        sampled_latent_space_16d = latent_space_16d[idx]
        sampled_filtered_conditions = filtered_conditions[idx]
    else:
        sampled_latent_space_16d = latent_space_16d
        sampled_filtered_conditions = filtered_conditions

    print(f"Sampled 16D Latent Space Shape: {sampled_latent_space_16d.shape}")
    print(f"Sampled Conditions Shape: {sampled_filtered_conditions.shape}")

    print("\nFitting UMAP on 16D latent representations...")
    umap_reducer = umap.UMAP(n_neighbors=70, min_dist=0.15, n_components=2, metric='cosine', random_state=42)
    latent_2d_umap = umap_reducer.fit_transform(sampled_latent_space_16d)
    print(f"UMAP 16D Latent Space Shape: {latent_2d_umap.shape}")

    # Compare the difference between UMAP on 16D latent space only OR 16D + conditions

    # Plot the UMAP of the 16D latent space for each condition
    for condition in condition_names:
        plt.figure(figsize=(8, 6))
        condition_index = condition_indices[condition]

        plt.scatter(
            latent_2d_umap[:, 0],
            latent_2d_umap[:, 1],
            c=sampled_filtered_conditions[:, condition_index],
            cmap=custom_cmap,
            alpha=0.7,
            edgecolors='black'
        )
        plt.colorbar(label=condition)
        plt.title(f'2D Visualization of 16D Latent Space Using UMAP Colored by {condition}')
        plt.xlabel('UMAP Dimension 1')
        plt.ylabel('UMAP Dimension 2')
        plt.grid(True)
        plt.show()


    print("\nCombining latent representations with conditions...")
    combined_data = np.concatenate([sampled_latent_space_16d, sampled_filtered_conditions], axis=1)
    print(f"Combined Data Shape: {combined_data.shape}")

    print("\nFitting UMAP on combined latent representations and conditions...")
    combined_2d_umap = umap_reducer.fit_transform(combined_data)
    print(f"UMAP Combined Data Shape: {combined_2d_umap.shape}")

    # Plot the UMAP of the combined latent space + conditions for each condition
    for condition in condition_names:
        plt.figure(figsize=(8, 6))
        condition_index = condition_indices[condition]

        plt.scatter(
            combined_2d_umap[:, 0],
            combined_2d_umap[:, 1],
            c=sampled_filtered_conditions[:, condition_index],
            cmap=custom_cmap,
            alpha=0.7,
            edgecolors='black'
        )
        plt.colorbar(label=condition)
        plt.title(f'2D Visualization of 16D Latent Space + Conditions Using UMAP Colored by {condition}')
        plt.xlabel('UMAP Dimension 1')
        plt.ylabel('UMAP Dimension 2')
        plt.grid(True)
        plt.show()

In [ ]:
# Check if wafertype removal is necessary -- remove the wafertype columns (indices 3, 4, 5) from filtered conditions
wafertype_indices = [3, 4, 5]
filtered_conditions_no_wafertype = np.delete(filtered_conditions, wafertype_indices, axis=1)

print(f"Filtered Conditions Shape (before removing wafertype): {filtered_conditions.shape}")
print(f"Filtered Conditions Shape (after removing wafertype): {filtered_conditions_no_wafertype.shape}")

# Update condition names
condition_names = [
    "Eta",                # Index 0
    "Wafer V",            # Index 1
    "Wafer U",            # Index 2
    "SumCALQ",            # Index 3
    "Layer Normalized"    # Index 4
]

condition_indices = {name: idx for idx, name in enumerate(condition_names)}
print("\nUpdated Condition indices mapping:")
print(condition_indices)

# Concatenate latent space with filtered conditions (with no wafertype)
combined_data = np.concatenate([latent_space_16d, filtered_conditions_no_wafertype], axis=1)
print(f"Combined Data Shape: {combined_data.shape}")

plt.figure(figsize=(15, 12))

colors = ['skyblue', 'orange', 'green', 'pink', 'cyan']

for i, name in enumerate(condition_names):
    plt.subplot(3, 2, i + 1)
    plt.hist(filtered_conditions_no_wafertype[:, i], bins=50, color=colors[i], edgecolor='black', alpha=0.7)
    plt.title(f"{name} Distribution (Filtered)")
    plt.xlabel(name)
    plt.ylabel("Frequency")
    plt.grid(True)

plt.tight_layout()
plt.show()


sample_size = 5000
if combined_data.shape[0] > sample_size:
    idx = np.random.choice(combined_data.shape[0], sample_size, replace=False)
    sampled_combined_data = combined_data[idx]
    sampled_filtered_conditions_no_wafertype = filtered_conditions_no_wafertype[idx]
else:
    sampled_combined_data = combined_data
    sampled_filtered_conditions_no_wafertype = filtered_conditions_no_wafertype

print(f"Sampled Combined Data Shape: {sampled_combined_data.shape}")
print(f"Sampled Conditions Shape (no wafertype): {sampled_filtered_conditions_no_wafertype.shape}")

# Fit UMAP on combined latent representations and conditions (without wafertype)
print("\nFitting UMAP on sampled combined latent representations and conditions...")
umap_reducer = umap.UMAP(n_neighbors=70, min_dist=0.15, n_components=2, metric='cosine', random_state=42)
combined_2d_umap = umap_reducer.fit_transform(sampled_combined_data)
print(f"UMAP Combined Data Shape: {combined_2d_umap.shape}")

for condition in condition_names:
    plt.figure(figsize=(8, 6))
    condition_index = condition_indices[condition]

    plt.scatter(
        combined_2d_umap[:, 0],
        combined_2d_umap[:, 1],
        c=sampled_filtered_conditions_no_wafertype[:, condition_index],
        cmap=custom_cmap,
        alpha=0.7,
        edgecolors='black'
    )
    plt.colorbar(label=condition)
    plt.title(f'2D Visualization of 16D Latent Space + Conditions Using UMAP Colored by {condition}')
    plt.xlabel('UMAP Dimension 1')
    plt.ylabel('UMAP Dimension 2')
    plt.grid(True)
    plt.show()

In [ ]:
single_wafer = inputs[234]
single_condition = conditions[234]

# Expand dimensions to match the expected input shape of the encoder
single_wafer_expanded = np.expand_dims(single_wafer, axis=0)          #shape: (1, 8, 8)
single_wafer_expanded = np.expand_dims(single_wafer_expanded, axis=-1)  # (1, 8, 8, 1)
single_condition_expanded = np.expand_dims(single_condition, axis=0)  # (1, 8)

single_latent = encoder.predict([single_wafer_expanded, single_condition_expanded])

print(f"Shape of single wafer: {single_wafer.shape}")
print(f"Latent representation: {single_latent}")

from matplotlib.patches import RegularPolygon
from matplotlib import cm
from matplotlib.colors import Normalize

single_wafer_2d = single_wafer.squeeze()  # Convert to (8, 8)

# Hexagonal grid
fig, ax = plt.subplots(figsize=(8, 8))

hex_radius = 1.0  # Radius of each hexagon
hex_height = np.sqrt(3) * hex_radius  # Height of each hexagon

# Normalization object for color mapping
norm = Normalize(vmin=np.min(single_wafer_2d), vmax=np.max(single_wafer_2d))

for i in range(single_wafer_2d.shape[0]):
    for j in range(single_wafer_2d.shape[1]):
        x = j * hex_height
        y = i * 1.5 * hex_radius  # y-coordinate for the hexagon for staggered rows

        # Offset ever other row
        if i % 2 == 1:
            x += hex_height / 2

        # Value for the current trigger cell
        value = single_wafer_2d[i, j]

        # Hexagon with the corresponding color and no edge
        hexagon = RegularPolygon((x, y), numVertices=6, radius=hex_radius,
                                 facecolor=cm.viridis(norm(value)), edgecolor=None)
        ax.add_patch(hexagon)


ax.set_xlim(1.1 * -hex_height, hex_height * (single_wafer_2d.shape[1] + 0.5))
ax.set_ylim(1.4 * -hex_radius, 1.4 * hex_radius * (single_wafer_2d.shape[0] + 0.5))
ax.set_aspect('equal')

sm = cm.ScalarMappable(cmap='viridis', norm=norm)
cbar = plt.colorbar(sm, ax=ax, label='Active Trigger Cells', shrink=0.6)

plt.title("Single Wafer Grid")
plt.axis('off')
plt.show()

# Another hexagonal grid visualization (not as good)
fig, ax = plt.subplots(figsize=(8, 8))

for i in range(8):
    for j in range(8):

        x = j + 0.5 * (i % 2)
        y = i * np.sqrt(3)/2

        # Check if this position is active (valid trigger cell) or inactive
        value = single_wafer[i, j]

        if value != 0:  # Assume non-zero values indicate active trigger cells (this is not the case right now)
            color = cm.viridis(value)
        else:
            color = "lightgray"

        hexagon = RegularPolygon((x, y), numVertices=6, radius=0.5, facecolor=color, edgecolor='black')
        ax.add_patch(hexagon)

ax.set_xlim(-1, 8)
ax.set_ylim(-1, 8 * np.sqrt(3)/2)


sm = cm.ScalarMappable(cmap='viridis')
sm.set_array(single_wafer)
cbar = plt.colorbar(sm, ax=ax, label='Active Trigger Cells', shrink=1.4)

plt.title("Single Wafer Grid (64 Positions)")
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()


# Simple 8x8 wafer plot
plt.figure(figsize=(6, 6))
plt.imshow(single_wafer.squeeze(), cmap='viridis', interpolation='none')
plt.colorbar(label='Active Trigger Cells')
plt.title("Single Wafer 8x8")
plt.gca().invert_yaxis()
plt.show()

umap_reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, n_components=2, metric='cosine', random_state=42)
latent_2d = umap_reducer.fit_transform(single_latent)

# Plot latent point (to check if single wafer)
plt.figure(figsize=(6, 6))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c='red', marker='o', edgecolor='black')
plt.title("Single Wafers Latent Space")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.grid(True)
plt.show()